In [1]:
#We import the necessary libraries.
%matplotlib inline

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import optimize
from importlib import reload

In [3]:
#We also import Juliet for the fitting process.
import juliet

In [4]:
#We also define a useful function to calculate the Median, Lower and Upper Errors of given parameters posterior.
def get_vals(vec):
    fvec   = np.sort(vec)

    fval  = np.median(fvec)
    nn = int(np.around(len(fvec)*0.15865))

    vali,valf = fval - fvec[nn],fvec[-nn] - fval
    return fval,vali,valf

In [5]:
#We first import the NEID data.
df = pd.read_csv("TIC 258804746_rv_unbin_newest.csv")

df

,bjd,bjdnonweighted,deltamin,rv,e_rv,pre_rv,pre_e_rv,sn18,sn102,exptime,...,NaD2_ind_e,PaD_ind_v,PaD_ind_e,NaNIR_ind_v,NaNIR_ind_e,rvccfsum,rvccfsumerr,rvccfmod,rvccfmoderr,driftfun
0,2.460662e+06,2.460662e+06,0.173601,-8.980639,2.618923,NaN,NaN,13.223364,28.454166,299.842613,...,0.004771,0.862367,0.029393,0.178822,0.003329,12.628763,0.003462,12.630490,0.003437,dailymodel0
1,2.460662e+06,2.460662e+06,-0.114278,-21.921024,3.141299,NaN,NaN,10.862042,24.040826,299.837112,...,0.005885,0.885610,0.036821,0.175247,0.003900,12.607195,0.003987,12.608372,0.003944,dailymodel0
2,2.460664e+06,2.460664e+06,-0.096900,-1.938046,2.359089,NaN,NaN,14.915557,31.877791,299.796731,...,0.004140,0.857568,0.024962,0.177756,0.002934,12.631427,0.003232,12.631962,0.003183,dailymodel0
3,2.460665e+06,2.460665e+06,-0.003376,6.412131,1.850221,NaN,NaN,19.394139,38.013901,299.791047,...,0.003100,0.853720,0.019657,0.181766,0.002430,12.638462,0.002631,12.639689,0.002613,dailymodel0
4,2.460666e+06,2.460666e+06,0.016561,9.844131,1.807651,NaN,NaN,20.013067,38.943889,299.809944,...,0.003036,0.862761,0.019464,0.178579,0.002403,12.643550,0.002598,12.644707,0.002576,dailymodel0
5,2.460681e+06,2.460681e+06,0.016067,-12.130043,4.233394,NaN,NaN,7.815653,19.082133,299.825572,...,0.008417,0.871790,0.048111,0.182076,0.005145,12.614045,0.004906,12.616045,0.004879,dailymodel0
6,2.460681e+06,2.460681e+06,-0.008045,6.774739,2.815522,NaN,NaN,12.345267,27.526787,299.765262,...,0.005170,0.831517,0.030800,0.176577,0.003514,12.632754,0.003691,12.633343,0.003639,dailymodel0
7,2.460686e+06,2.460686e+06,0.040945,4.610357,5.800396,NaN,NaN,5.388136,14.032025,299.821304,...,0.012453,0.891774,0.073522,0.177412,0.007154,12.639205,0.006070,12.639453,0.006092,dailymodel0
8,2.460726e+06,2.460726e+06,0.058585,-1.362416,2.957234,NaN,NaN,11.574093,26.713125,299.789961,...,0.005486,0.868275,0.031035,0.179768,0.003662,12.624598,0.003848,12.625768,0.003802,dailymodel0
9,2.460726e+06,2.460726e+06,-0.012008,-6.294036,2.703145,NaN,NaN,12.760667,28.473569,300.003695,...,0.004934,0.896557,0.028888,0.174780,0.003440,12.617792,0.003598,12.619572,0.003559,dailymodel0


In [7]:
rv = df["rv"]
print(np.std(rv))

9.675515800726922


In [6]:
#We take the timestamps (BJD) and the RV Errors from the dataframe.
times = df["bjd"]
rv_err = df["e_rv"]

print(f"Timestamps: \n{times}")
print(f"\nRV Errors: \n{rv_err}")

Timestamps: 
0     2.460662e+06
1     2.460662e+06
2     2.460664e+06
3     2.460665e+06
4     2.460666e+06
5     2.460681e+06
6     2.460681e+06
7     2.460686e+06
8     2.460726e+06
9     2.460726e+06
10    2.460726e+06
11    2.460726e+06
Name: bjd, dtype: float64

RV Errors: 
0     2.618923
1     3.141299
2     2.359089
3     1.850221
4     1.807651
5     4.233394
6     2.815522
7     5.800396
8     2.957234
9     2.703145
10    5.567126
11    5.375776
Name: e_rv, dtype: float64


In [7]:
#To define a Keplerian Model, we can use Radvel.
import radvel

In [8]:
#We define a function that samples the parameters for the planet TOI-2431 b and the injected planet "TOI-2431 c".
def planet_injector(no_of_injections):
    """Takes the number of injections the user wants to do and returns the sampled parameter set.
    
    Input:
    ----------------------------------------------------------------------------
    no_of_injections: The number of planets we will inject into the RV data
    
    Output:
    ----------------------------------------------------------------------------
    df_planet: Pandas Dataframe containing the parameters of the planets
    
    """    
    #We define the orbital parameters of Planet TOI-2431 b.
        #Period (P): Normal Distribution, taken from the TESS fit.
        #T0 (T0): Normal distribution, taken from the TESS fit.
        #Eccentricity: Kept to zero, assuming circular orbit.
        #Argument of Periapsis (omega): Kept as 90 degrees, but since radvel wants it in radians, π/2.
        #Semi Amplitude of RV (K): Uniform Distribution, sampled from 0 to 50 m/s randomly.
    P_b = np.random.normal(0.2241957793, 4e-7, no_of_injections)
    T0_b = np.random.normal(2460258.8685502028, 0.00015, no_of_injections)
    ecc_b = np.full(no_of_injections, 0)
    omega_b = np.full(no_of_injections, np.pi/2)
    K_b = np.random.uniform(0, 50, no_of_injections)
    
    #We define the orbital parameters of the "Injected Planet" TOI-2431 c.
        #Period (P): Uniform Distribution, sampled from 1 day to 100 days.
        #T0 (T0): Uniform Distribution, sampled from -P/2 to P/2.
        #Eccentricity: Kept to zero, assuming circular orbit.
        #Argument of Periapsis (omega): Kept as 90 degrees, but since radvel wants it in radians, π/2.
        #Semi Amplitude of RV (K): Uniform Distribution, sampled from 0 to 20 m/s randomly.
    P_c = np.random.uniform(1, 100, no_of_injections)
    T0_c = np.random.uniform(-P_c/2, P_c/2, no_of_injections)
    ecc_c = np.full(no_of_injections, 0)
    omega_c = np.full(no_of_injections, np.pi/2)
    K_c = np.random.uniform(0, 50, no_of_injections)

    #We create a Pandas Dataframe that contains the parameters of our planets.
    df_planet = pd.DataFrame({
                "Planet 1": "TOI-2431 b",
                "P_b (days)": P_b,
                "T0_b": T0_b,
                "ecc_b": ecc_b,
                "omega_b (rad)": omega_b,
                "K_b (m/s)": K_b,
                "Planet 2": "TOI-2431 c",
                "P_c (days)": P_c,
                "T0_c": T0_c,
                "ecc_c": ecc_c,
                "omega_c (rad)": omega_c,
                "K_c (m/s)": K_c
                })

    return df_planet

df_planet = planet_injector(no_of_injections = 5)

In [9]:
#Now we call in the Keplerian Model from radvel.
help(radvel.kepler.rv_drive)

Help on function rv_drive in module radvel.kepler:

rv_drive(t, orbel, use_c_kepler_solver=True)
    RV Drive
    Args:
        t (array of floats): times of observations
        orbel (array of floats): [per, tp, e, om, K].            Omega is expected to be            in radians
        use_c_kepler_solver (bool): (default: True) If             True use the Kepler solver written in C, else             use the Python/NumPy version.
    Returns:
        rv: (array of floats): radial velocity model



In [10]:
#We import the following library to remove the previous results from previous runs.
import shutil

#We then create the RV datapoints for given timestamps in a for loop, going through each sampling example.
for i in range(len(df_planet)):
    #We first remove the previous results if there are any.
    shutil.rmtree("toi_2431_fit", ignore_errors = True)
    
    #We create the synthetic RV datapoints for given timestamps for TOI-2431 b.
    RV_data_b = radvel.kepler.rv_drive(times, [df_planet["P_b (days)"].iloc[i], df_planet["T0_b"].iloc[i], 
                                               df_planet["ecc_b"].iloc[i], df_planet["omega_b (rad)"].iloc[i], 
                                               df_planet["K_b (m/s)"].iloc[i]], use_c_kepler_solver = True)

    #We create the synthetic RV datapoints for given timestamps for TOI-2431 c.
    RV_data_c = radvel.kepler.rv_drive(times, [df_planet["P_c (days)"].iloc[i], df_planet["T0_c"].iloc[i], 
                                               df_planet["ecc_c"].iloc[i], df_planet["omega_c (rad)"].iloc[i], 
                                               df_planet["K_c (m/s)"].iloc[i]], use_c_kepler_solver = True)

    #We then combine these RV datapoints to create the combined synthetic RV data.
    RV_data = RV_data_b + RV_data_c

    #We then add the synthetic RV data the RV error of our instrument to include the Errors we would expect if it 
    #were real data.
    for x in range(len(RV_data)):
        RV_data[x] = RV_data[x] + np.random.normal(0, rv_err[x], 1)

    print(80 * "-")
    print(RV_data)
    print(80 * "-")

    #We then define a dataframe for the current RV data we are working with.
    neid = pd.DataFrame()

    #We add the current RV data we have on this iteration.
    neid["bjd"] = times
    neid["rv"] = RV_data
    neid["e_rv"] = rv_err

    #We separate them per night to implement the Floating Chunk Offset (FCO) method.
    neid["night"] = neid["bjd"].astype(int)

    nights = {}
    for n, (night, group) in enumerate(neid.groupby("night"), start=1):
        nights[n] = group

    #We then create DataFrames with the different nights.
    selected_nights = [n for n, group in nights.items() if len(group) > 1]
    dataframes = {f"df_{j+1}": nights[n] for j, n in enumerate(selected_nights)}

    #We create juliet dictionaries for the current iteration.
    times_rvs, rvs, rvs_err = {}, {}, {}

    #We then implement FCO method.
    for j in range(1,len(dataframes)+1):
        times_rvs[f"NEID{j}"], rvs[f"NEID{j}"], rvs_err[f"NEID{j}"] = dataframes[f"df_{j}"].bjd.values, dataframes[f"df_{j}"].rv.values, dataframes[f"df_{j}"].e_rv.values

    #We then run the Juliet using the Priors we have.
    dataset = juliet.load(priors="../../data/priors/TOI-2431_priors_synthetic_rv.dat", t_rv=times_rvs, y_rv=rvs, yerr_rv=rvs_err,out_folder=f"toi_2431_fit_{i}")
        
    results = dataset.fit(sampler = "dynamic_dynesty", n_live = 2500, resume = False)

    #After the run, we can use the results to check whether FCO has actually managed to capture the underlying
    #truth or not by checking the Semi-Amplitude of the RV curve.
    print(get_vals(results.posteriors["posterior_samples"]["K_p1"]))
    K_truth = df_planet["K_b (m/s)"].iloc[i]
    K_fit = juliet.utils.get_quantiles(results.posteriors['posterior_samples']['K_p1'])[0]
    K_lower_err = get_vals(results.posteriors['posterior_samples']['K_p1'])[1]
    K_upper_err = get_vals(results.posteriors['posterior_samples']['K_p1'])[2]

    #We define the 1-sigma range of the injected K value as follows.
    lower_bound = K_fit - K_lower_err
    upper_bound = K_fit + K_upper_err
    
    #We then check whether the K_fit is in 1-sigma range of the injected K value, K_truth.
    print(f"K_truth: {K_truth}")
    print(f"K_fit: {K_fit}")
    df_planet.loc[i, "K_truth (m/s)"] = K_truth
    df_planet.loc[i, "K_fit (m/s)"] = K_fit
    
    if lower_bound <= K_truth <= upper_bound:
        df_planet.loc[i, "Result"] = "Recovered"
        print("Accepted!")

    else:
        df_planet.loc[i, "Result"] = "Not Recovered"
        print("Denied!")

--------------------------------------------------------------------------------
0      9.769354
1    -61.276759
2    -74.044844
3     -0.486709
4    -54.449437
5    -15.620008
6     67.643286
7     74.744146
8     -1.106094
9     -1.609791
10   -57.673042
11   -53.411001
Name: bjd, dtype: float64
--------------------------------------------------------------------------------


17155it [02:44, 104.56it/s, batch: 5 | bound: 33 | nc: 1 | ncall: 477856 | eff(%):  3.477 | loglstar: -24.837 < -18.588 < -19.487 | logz: -34.015 +/-  0.122 | stop:  0.945]            


(41.78790110706395, 1.6743851303116628, 1.6836438776636484)
K_truth: 42.468143076171295
K_fit: 41.78790110706395
Accepted!
--------------------------------------------------------------------------------
0     48.191331
1      9.822398
2     -0.872068
3     37.623130
4      9.157828
5    -27.954228
6     15.504191
7     -0.207789
8     39.325549
9     37.015518
10     8.900167
11    -0.353986
Name: bjd, dtype: float64
--------------------------------------------------------------------------------


16895it [02:38, 106.72it/s, batch: 5 | bound: 32 | nc: 1 | ncall: 470319 | eff(%):  3.478 | loglstar: -24.284 < -18.197 < -19.022 | logz: -33.150 +/-  0.119 | stop:  0.969]            


(23.289002546880052, 1.6003826255416058, 1.6893360962229842)
K_truth: 23.241094533301656
K_fit: 23.289002546880052
Accepted!
--------------------------------------------------------------------------------
0     28.816435
1    -48.454353
2    -73.917823
3     24.598696
4      3.068515
5    -75.772259
6     16.400875
7     37.277998
8     20.345615
9     13.921320
10   -43.319856
11   -38.113829
Name: bjd, dtype: float64
--------------------------------------------------------------------------------


...

(45.62914145123783, 1.715730320148289, 1.7070936867072959)
K_truth: 46.86720941290255
K_fit: 45.629380424618645
Accepted!
--------------------------------------------------------------------------------
0     43.064260
1     -8.058332
2     -3.228754
3     48.363856
4     21.460106
5     -4.523602
6     43.341118
7     19.664266
8     56.161323
9     57.959463
10    29.715577
11    25.057963
Name: bjd, dtype: float64
--------------------------------------------------------------------------------


18949it [03:16, 96.37it/s, batch: 5 | bound: 37 | nc: 1 | ncall: 534502 | eff(%):  3.445 | loglstar: -27.786 < -20.824 < -21.940 | logz: -36.015 +/-  0.112 | stop:  0.787]             


(24.979267794859894, 2.549972075893546, 2.243931224139434)
K_truth: 24.004368799402485
K_fit: 24.979267794859894
Accepted!
--------------------------------------------------------------------------------
0     24.943457
1    -16.814431
2    -25.297725
3     18.757112
4    -14.890138
5    -30.858879
6     25.588814
7     13.307252
8     16.375602
9     15.689051
10   -29.355568
11   -14.698977
Name: bjd, dtype: float64
--------------------------------------------------------------------------------


16556it [02:40, 103.24it/s, batch: 4 | bound: 33 | nc: 1 | ncall: 460711 | eff(%):  3.477 | loglstar: -26.522 < -20.174 < -21.093 | logz: -35.046 +/-  0.121 | stop:  0.963]             


(27.146495941195894, 1.677520083960225, 1.7335341111341975)
K_truth: 24.905892570405687
K_fit: 27.147510715968828
Denied!


In [13]:
print(df_planet)

     Planet 1  P_b (days)          T0_b  ecc_b  omega_b (rad)  K_b (m/s)  \
0  TOI-2431 b    0.224196  2.460259e+06      0       1.570796  42.468143   
1  TOI-2431 b    0.224196  2.460259e+06      0       1.570796  23.241095   
2  TOI-2431 b    0.224196  2.460259e+06      0       1.570796  46.867209   
3  TOI-2431 b    0.224196  2.460259e+06      0       1.570796  24.004369   
4  TOI-2431 b    0.224196  2.460259e+06      0       1.570796  24.905893   

     Planet 2  P_c (days)       T0_c  ecc_c  omega_c (rad)  K_c (m/s)  \
0  TOI-2431 c   59.353119 -17.501493      0       1.570796  36.723554   
1  TOI-2431 c   80.551630 -15.577205      0       1.570796  32.035983   
2  TOI-2431 c    2.771613   0.672144      0       1.570796  32.664515   
3  TOI-2431 c   54.559570  -5.603130      0       1.570796  41.161214   
4  TOI-2431 c   35.758925   0.017088      0       1.570796   1.010952   

   K_truth (m/s)  K_fit (m/s)         Result  
0      42.468143    41.787901      Recovered  
1      23.

In [15]:
df_planet

,Planet 1,P_b (days),T0_b,ecc_b,omega_b (rad),K_b (m/s),Planet 2,P_c (days),T0_c,ecc_c,omega_c (rad),K_c (m/s),K_truth (m/s),K_fit (m/s),Result
0,TOI-2431 b,0.224196,2.460259e+06,0,1.570796,42.468143,TOI-2431 c,59.353119,-17.501493,0,1.570796,36.723554,42.468143,41.787901,Recovered
1,TOI-2431 b,0.224196,2.460259e+06,0,1.570796,23.241095,TOI-2431 c,80.551630,-15.577205,0,1.570796,32.035983,23.241095,23.289003,Recovered
2,TOI-2431 b,0.224196,2.460259e+06,0,1.570796,46.867209,TOI-2431 c,2.771613,0.672144,0,1.570796,32.664515,46.867209,45.629380,Recovered
3,TOI-2431 b,0.224196,2.460259e+06,0,1.570796,24.004369,TOI-2431 c,54.559570,-5.603130,0,1.570796,41.161214,24.004369,24.979268,Recovered
4,TOI-2431 b,0.224196,2.460259e+06,0,1.570796,24.905893,TOI-2431 c,35.758925,0.017088,0,1.570796,1.010952,24.905893,27.147511,Not Recovered


In [14]:
#df_planet.to_csv("df_planet.csv", index=False)